In [ ]:
%pip install fiona

In [3]:
import os.path
import fiona
from shapely.geometry import shape
import csv
from os import path

class ShapefileValidator:
    def __init__(self):
        self.sf = None
        
    def run(self):
        # Get the shapefile from user
        shapefile_path = input("Enter the shapefile path: ")
        if not os.path.exists(shapefile_path):
            print("File does not exist.")
            return
        
        # Open the shapefile
        self.sf = fiona.open(shapefile_path)
            
        print("""Options:
            1. Validate
            2. Check intersection
            3. Remove invalid geometry and export to shapefile
            4. Remove intersecting geometry and export to shapefile
            5. Convert to CSV""")
        
        option = int(input("Enter option number: "))
        if option == 1:
            self.validate()
        elif option == 2:
            self.check_intersection()
        elif option == 3:
            filename, path = self.get_output_path()
            self.remove_invalid_geometry(path, filename)
        elif option == 4:
            filename, path = self.get_output_path()
            self.remove_intersecting_geometry(path, filename)
        elif option == 5:
            filename, path = self.get_output_path()
            self.convert_to_csv(path, filename)
        else:
            print("Invalid option number.")


    def get_output_path(self):
        filename = input("Enter output filename: ")
        path = input("Enter output directory path: ")
        return filename, path
    
    def validate(self):
        # If no shapefile is loaded
        if not self.sf:
            print("No shapefile loaded.")
            return
        
        # Errors list to keep track of any invalid features found in the shapefile
        errors = []

        # Iterate over all shapes in search for the geo_interface attribute.
        # If attribute not found, shape in shapefile is invalid and append to 'errors'
        for feature in self.sf:
            try:
                feature['__geo_interface__']
            except:
                errors.append(feature['id'])
        
        if errors:
            print(f"{len(errors)} invalid features found:")
            for error in errors:
                print(f"Feature {error}")
        else:
            print("All features are valid.")
    
    def check_intersection(self):
        if not self.sf:
            print("No shapefile loaded.")
            return
        
        # Check for the type of shapefile. 
        shape_type = self.sf.meta['schema']['geometry']
        if shape_type in ['MultiPoint', 'Point', 'MultiLineString', 'LineString']:
            print("Intersection check is not applicable to this shapefile type.")
            return
        
        errors = []
        # Loop through all shape pairs and check for intersections. 
        # Enumerate gets the index of the first shape, shapes method gets the second shape.
        for i, feature1 in enumerate(self.sf):
            shape1 = shape(feature1['geometry'])
            for feature2 in self.sf[i+1:]:
                shape2 = shape(feature2['geometry'])
                if shape1.type == shape2.type and shape1.intersects(shape2):
                    errors.append((feature1['id'], feature2['id']))
        
        if errors:
            print(f"{len(errors)} intersecting features found:")
            for error in errors:
                print(f"Features {error[0]} and {error[1]}")
        else:
            print("No intersecting features found.")
    
    def remove_invalid_geometry(self, output_path, output_filename):
        if not self.sf:
            print("No shapefile loaded.")
            return
        
        invalid_indices = []
        for i, feature in enumerate(self.sf):
            try:
                shape(feature['geometry']).__geo_interface__
            except:
                invalid_indices.append(i)
        
        if invalid_indices:
            print(f"{len(invalid_indices)} invalid features found. Removing them...")
            output_shapefile = fiona.open(output_path + output_filename + '.shp', 'w', **self.sf.meta)
            for i, feature in enumerate(self.sf):
                if i not in invalid_indices:
                    output_shapefile.write(feature)
            output_shapefile.close()
            print("Shapefile exported.")
        else:
            print("No invalid features found. Nothing to export.")
    
    def remove_intersecting_geometry(self, output_path, output_filename):
        from shapely.geometry import shape
        if not self.sf:
            print("No shapefile loaded.")
            return
        
        shape_type = self.sf.meta['schema']['geometry']
        
        if shape_type in ['MultiPoint', 'Point', 'MultiLineString', 'LineString']:
            print("Intersection check is not possible")
            return
        
        errors = []
        for i, shape1 in enumerate(self.sf):
            for shape2 in self.sf[i+1:]:
                if shape1['geometry']['type'] == shape2['geometry']['type'] and shape(shape1['geometry']).intersects(shape(shape2['geometry'])):
                    errors.append((shape1['id'], shape2['id']))

        if errors:
            # If there are any errors, remove them.
            invalid_indices = set([i for e in errors for i in e])
            output_shapefile = fiona.open(output_path + output_filename + '.shp', 'w', **self.sf.meta)
            
            with output_shapefile:
                for i, shape in enumerate(self.sf):
                    if i not in invalid_indices:
                        output_shapefile.write(shape)

            print("Shapefile exported.")
        else:
            print("No intersecting features found.")

    
    def convert_to_csv(self, output_path, output_filename):
        if not self.sf:
            print("No shapefile loaded.")
            return

        # Load the shapefile
        features = [feature for feature in self.sf]

        with open(output_path + output_filename + '.csv', 'w', newline='') as csvfile:
            writer = csv.writer(csvfile)

            # Write the header row with field names
            fields = features[0]['properties'].keys()
            writer.writerow(fields)

            # Write each feature as a row in the CSV file
            for feature in features:
                row = [feature['properties'][field] for field in fields]
                writer.writerow(row)

        print("CSV file exported.")

In [5]:
validator = ShapefileValidator()
validator.run()

Options:
            1. Validate
            2. Check intersection
            3. Remove invalid geometry and export to shapefile
            4. Remove intersecting geometry and export to shapefile
            5. Convert to CSV
CSV file exported.
